<a href="https://colab.research.google.com/github/chinchinXD/Codes/blob/main/merged_cwv_and_reports.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## **Adding error reports to main file**

In [ ]:
import pandas as pd
from google.colab import files

# Upload the error files
uploaded_files = files.upload()

In [ ]:
# Process the uploaded files
error_files = {}
for file_name, file_content in uploaded_files.items():
    error_files[file_name] = file_content

In [ ]:
import os
main_sheet_file = files.upload()
# Save the main sheet file to the local filesystem
main_sheet_filename = next(iter(main_sheet_file))
main_sheet_path = os.path.join('./', main_sheet_filename)
with open(main_sheet_path, 'wb') as f:
    f.write(main_sheet_file[main_sheet_filename])
# Read the main sheet
main_sheet = pd.read_excel(main_sheet_path)

Saving FD pages test.xlsx to FD pages test.xlsx


In [ ]:
# Process each error file
for error_file_name, error_file_content in error_files.items():
    error_file_data = pd.read_excel(pd.ExcelFile(error_file_content), sheet_name='Table')
    column_name = error_file_name.split('-')[0].strip()

    # Iterate over each URL in the main file
    for url in main_sheet['URL']:
        found = False

        # Iterate over each URL in the error file
        for error_url in error_file_data['URL']:
            if url == error_url:
                found = True
                break

        # Update the main file with the error file name if URL found
        if found:
            main_sheet.loc[main_sheet['URL'] == url, column_name] = error_file_name

    # If no match found for any URL, update with 'Good'
    main_sheet[column_name].fillna('Good', inplace=True)

# Save the updated main file with error columns
output_file_path = '/content/output_file.xlsx'
main_sheet.to_excel(output_file_path, index=False)

# Download the output file
files.download(output_file_path)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

## **CORE WEB VITALS**

In [ ]:
import pandas as pd
import requests
from concurrent.futures import ThreadPoolExecutor
from google.colab import files

# Read the Excel file and extract URLs from the "URL" column
df = main_sheet
urls = df["URL"].tolist()

In [ ]:
# Define the API endpoint
api_url = "https://www.googleapis.com/pagespeedonline/v5/runPagespeed"

# Function to fetch metrics for a given URL and strategy
def fetch_metrics(url):
    params = {
        "url": url,
        "strategy": "mobile",
        "category": "performance",
        "fields": "lighthouseResult"
    }
    response = requests.get(api_url, params=params)
    data = response.json()
    try:
        lcp = data["lighthouseResult"]["audits"]["largest-contentful-paint"]["displayValue"]
        fid = data["lighthouseResult"]["audits"]["max-potential-fid"]["displayValue"]
        cls = data["lighthouseResult"]["audits"]["cumulative-layout-shift"]["displayValue"]
        return url, lcp, fid, cls
    except KeyError:
        return url, "N/A", "N/A", "N/A"

# Create a ThreadPoolExecutor to execute requests concurrently
with ThreadPoolExecutor() as executor:
    # Submit tasks for fetching metrics
    tasks = [executor.submit(fetch_metrics, url) for url in urls]

    # Extract metrics
    results = [task.result() for task in tasks]

# Create a new DataFrame to store the results
results_df = pd.DataFrame(columns=["URL", "LCP (Mobile)", "FID (Mobile)", "CLS (Mobile)"])
for url, lcp_mobile, fid_mobile, cls_mobile in results:
    results_df = pd.concat([
        results_df,
        pd.DataFrame({"URL": [url], "LCP (Mobile)": [lcp_mobile], "FID (Mobile)": [fid_mobile], "CLS (Mobile)": [cls_mobile]})
    ], ignore_index=True)

# Merge the results with the original DataFrame based on the URL column
merged_df = pd.merge(df, results_df, on="URL", how="left")

# Save the merged DataFrame to a new Excel file
output_file = "added CWV.xlsx"
merged_df.to_excel(output_file, index=False)
files.download(output_file)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

## **Add performance metrics**

In [ ]:
#installtions
!pip install google-auth-oauthlib
!pip install google-api-python-client
!pip install oauth2client

In [ ]:
import pandas as pd
from datetime import datetime, timedelta
from google_auth_oauthlib.flow import InstalledAppFlow
import logging
from googleapiclient.discovery import build
from googleapiclient import discovery
import datetime
from datetime import date, timedelta
import httplib2
from googleapiclient.discovery import build
from oauth2client.client import OAuth2WebServerFlow
from collections import defaultdict
from dateutil import relativedelta
import argparse
from oauth2client import client
from oauth2client import file
from oauth2client import tools
import re
import os
from urllib.parse import urlparse
from google.oauth2 import service_account

In [ ]:
site = 'https://www.bajajfinserv.in/'  # Property to extract
num_days = 1  # Number of Days, Months to Extract
stype=["web", "image", "video", "news"]

maxRows = 25000
i = 0
output_rows1 = []
output_rows2 = []

start_date = datetime.datetime.strptime("2023-05-01", "%Y-%m-%d")
end_date = datetime.datetime.strptime("2023-06-06", "%Y-%m-%d")

#start_date=date.today()-relativedelta.relativedelta(days=3)
#end_date=date.today()-relativedelta.relativedelta(days=3)

In [ ]:
import json
scope = ['https://www.googleapis.com/auth/webmasters.readonly']
json_key_file='/content/json_key.json'
# Open the JSON file and read its contents
with open(json_key_file) as f:
    json_key = json.load(f)
credentials = service_account.Credentials.from_service_account_info(json_key, scopes=scope)
service = build(
    'webmasters',
    'v3',
    credentials=credentials
)

In [ ]:
def date_range(start_date, end_date, delta=timedelta(days=1)):
    """
    The range is inclusive, so both start_date and end_date will be returned
    Args:
        start_date: The datetime object representing the first day in the range.
        end_date: The datetime object representing the second day in the range.
        delta: A datetime.timedelta instance, specifying the step interval. Defaults to one day.
    Yields:
        Each datetime object in the range.
    """
    current_date = start_date
    while current_date <= end_date:
        yield current_date
        current_date += delta


In [ ]:
def extract_data_sa(stype):
    for date in date_range(start_date, end_date):
        date = date.strftime("%Y-%m-%d")
        print(date)
        i = 0
        while True:

            request = {
                'startDate' : date,
                'endDate' : date,
                'dimensions' : ["searchAppearance"],
                'searchType': stype,
                'rowLimit' : maxRows,
                'startRow' : i * maxRows

            }

            response = service.searchanalytics().query(siteUrl = site, body=request).execute()
            print()
            if response is None:
                print("there is no response")
                break
            if 'rows' not in response:
                print("row not in response")
                break
            else:
                for row in response['rows']:
                    searchAppearance = row['keys'][0]
                    output_row = [date,searchAppearance,stype,row['clicks'], row['ctr'], row['impressions'], row['position']]
                    output_rows1.append(output_row)
                i = i + 1

    df = pd.DataFrame(output_rows1, columns=['date','search_appearance','search_type','clicks','ctr','impressions','position'])
    return df

In [ ]:
df2=pd.DataFrame()
for st in stype:
    df2 =df2.append(extract_data_sa(st))
df2.drop_duplicates(inplace=True)
lst_search_appearance =list(set(df2['search_appearance'].to_list()))

In [ ]:
def extract_data(sAppear):
    for date in date_range(start_date, end_date):
        date = date.strftime("%Y-%m-%d")
        print(date)
        i = 0
        while True:

            request = {
                'startDate' : date,
                'endDate' : date,
                'searchType': "web",
                "dimensions":["query","page","country","device"],
                "dimensionFilterGroups":[
                    {
                        "filters": [
                            {
                            "dimension": "searchAppearance",
                            "operator": "equals",
                            "expression": sAppear
                            }

                        ]
                    }

                ],

                'rowLimit' : maxRows,
                'startRow' : i * maxRows
            }


            response = service.searchanalytics().query(siteUrl = site, body=request).execute()
            print()
            if response is None:
                print("there is no response")
                break
            if 'rows' not in response:
                print("row not in response")
                break
            else:
                for row in response['rows']:
                    keyword = row['keys'][0]
                    page = row['keys'][1]
                    country = row['keys'][2]
                    device = row['keys'][3]
                    IsSVDone = ''
                    KeywordID=''
                    output_row = [date, page,keyword,country,device,sAppear,row['clicks'], row['ctr'], row['impressions'], row['position'],IsSVDone,KeywordID]
                    output_rows2.append(output_row)
                i = i + 1

    dff = pd.DataFrame(output_rows2, columns=['date','page','query', 'country', 'device','search_appearance','clicks','ctr','impressions','position','IsSVDone','KeywordID'])
    return dff

In [ ]:
df3=pd.DataFrame()
for sa in lst_search_appearance:
    df3 =df3.append(extract_data(sa))
df3.drop_duplicates(inplace=True)
#df3.drop(df3[(df3['query'] == 'web')].index, inplace=True)

In [ ]:
#some changes
df3['clicks'] = df3['clicks'].astype('int')
df3['ctr'] = df3['ctr'] * 100
df3['impressions'] = df3['impressions'].astype('int')
df3['position'] = df3['position'].round(2)

In [ ]:
df3.head()

In [ ]:
# Read the main Excel file with "Sheet1"
main_file_path = "/content/your_file_with_metrics.xlsx"
main_file = pd.ExcelFile(main_file_path)
main_df_sheet1 = main_file.parse("Sheet1")

# Perform the matching and populate "Sheet2"
df3_matched = df3.merge(main_df_sheet1, left_on="page", right_on="URL", how="inner")
sheet2_data = df3_matched[["URL", "date", "page", "query", "country", "device", "search_appearance", "clicks", "ctr", "impressions", "position", "IsSVDone", "KeywordID"]]

# Append "Sheet2" to the main Excel file
with pd.ExcelWriter(main_file_path, mode="w") as writer:
    main_df_sheet1.to_excel(writer, sheet_name="Sheet1", index=False)
    sheet2_data.to_excel(writer, sheet_name="Sheet2", index=False)

# Download the updated main Excel file
files.download(main_file_path)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>